In [2]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [3]:
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

final_entities=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_entities_with_embeddings.csv",['entity_embedding'])
final_entities

,entity_name,entity_type,description,id,entity_embedding
0,49ers,ORGANIZATION,49ers is a football team,1,"[0.04144171625375748, -0.007414397317916155, -..."
1,Barbarians,CONCEPT,a character class in the game,2,"[0.0122455395758152, -0.007463064510375261, -0..."
2,Bills,ORGANIZATION,The team that Allen plays for,3,"[0.030726933851838112, 0.0030639127362519503, ..."
3,Bolt Mobility,COMPANY,Bolt Mobility is introducing distance-based pr...,4,"[0.004269156605005264, -0.011844240128993988, ..."
4,Crimson Tide,TEAM,Crimson Tide is the nickname of the Alabama fo...,5,"[0.004862545058131218, -0.003534935647621751, ..."
...,...,...,...,...,...
23565,家庭自动化,CONCEPT,家庭自动化是指通过网络连接和控制家庭设备的技术。,23569,"[0.02464906871318817, 0.004047214984893799, -0..."
23566,昂贵的,CONCEPT,昂贵的是形容 soundbar 的形容词,23570,"[0.04650348424911499, 0.02652094140648842, -0...."
23567,智能扬声器,CONCEPT,智能扬声器是一种可以通过网络连接和控制的扬声器。,23571,"[0.03134648874402046, 0.013866862282156944, -0..."
23568,杜比 atmos,CONCEPT,杜比 atmos is a surround sound technology.,23572,"[0.01925036683678627, 0.018349502235651016, -0..."


In [4]:
final_relationships=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_relationships_with_embeddings.csv",['relationship_embedding'])
final_relationships

,source_entity,target_entity,relationship_type,relationship_description,source_id,target_id,relationship_embedding
0,Li,Falun Gong,Affiliated-with,Li is affiliated with Falun Gong.,5841,3963,"[0.006164404563605785, 0.004094846546649933, -..."
1,Epoch Times,Li,Worked-for,Epoch Times worked for Li.,3835,5841,"[0.014970953576266766, -0.01761895790696144, 0..."
2,The Epoch Times,Location,Located-in,The Epoch Times is located in multiple countries.,9282,5932,"[0.022465074434876442, -0.007946091704070568, ..."
3,Microsoft,The Epoch Times,Affiliated-with,Microsoft is affiliated with The Epoch Times.,6510,9282,"[0.024022087454795837, -0.024629345163702965, ..."
4,California,The Epoch Times,Located-in,California is located in The Epoch Times.,2502,9282,"[0.02883162721991539, -0.014359557069838047, -..."
...,...,...,...,...,...,...,...
26033,Lille,Alphonso Davies,Part-of,Lille is a part of Davies,5873,1515,"[0.009421243332326412, 0.025672562420368195, -..."
26034,Canada Soccer,"November 18, 2023",Happened-at,"November 18, 2023 is an event that happened at...",2535,7125,"[0.05737772211432457, 0.003261846723034978, -0..."
26035,David,Lille,Part-of,David is a part of Lille,3263,5873,"[-0.006869114935398102, 0.004632634576410055, ..."
26036,David,Milan Borjan,Part-of,David is a part of Borjan,3263,6561,"[0.024512803182005882, -0.0016717436956241727,..."


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# 假设你使用 Sentence-BERT 作为嵌入模型
model = SentenceTransformer('intfloat/multilingual-e5-large')

# 将问题转换为嵌入向量
def get_query_embedding(query):
    query_embedding = model.encode(query)
    return query_embedding

# 计算查询嵌入和实体嵌入之间的余弦相似度
def find_k_nearest_entities(query_embedding, final_entities, k=3):
    # 过滤掉嵌入为空的情况
    valid_entities = final_entities.dropna(subset=['entity_embedding'])
    # 确保所有嵌入向量的维度一致
    entity_embeddings = np.array([emb for emb in valid_entities['entity_embedding'].values if len(emb) == len(query_embedding)])
    similarities = cosine_similarity([query_embedding], entity_embeddings)[0]
    valid_entities = valid_entities.loc[valid_entities['entity_embedding'].apply(lambda x: len(x) == len(query_embedding))]
    valid_entities['similarity'] = similarities
    nearest_entities = valid_entities.sort_values(by='similarity', ascending=False).head(k)
    return nearest_entities

# 获取查询嵌入
query = "Which company, recently scrutinized by TechCrunch for both its antitrust legal defense involving document disclosure and the alleged harm its AI has caused to news publishers' revenue, also claims that its new generative AI model Gemini has capabilities that rival or exceed those of OpenAI's models?"
query_embedding = get_query_embedding(query)

# 查询最相近的 k 个实体
k = 3
nearest_entities = find_k_nearest_entities(query_embedding, final_entities, k)
nearest_entities[['entity_name','description', 'similarity']]

,entity_name,description,similarity
14630,gemini,Gemini is a new flagship generative AI model f...,0.846514
7275,OpenAI,OpenAI is an artificial intelligence research ...,0.822309
21528,techcrunch,TechCrunch is a technology news website and me...,0.815579
